In [90]:
import pandas as pd
import numpy as np

file_path = '2020_LER_20200804_V006.xlsx'
datatype = 'n'
num_input=4
num_output=6
num_in_cycle=50
num_of_cycle=200
num_total = num_in_cycle * num_of_cycle
num_train=75
num_val=10
num_test=15
x_cols="B:G"
y_cols="H:P"
header=0


data_x = pd.read_excel('./'+file_path, sheet_name='Generated DATAs',usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('./'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=num_total+1, header=header)

X_all , Y_all = np.zeros((num_total, num_input+1)), np.zeros((num_total, num_output)) # onehot encoding
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input+1)), np.zeros((num_of_cycle, num_output))

# DATA_X preprocessing

# 1. N, P to 10, 01 (one-hot encoding)
data_x =pd.get_dummies(data_x, columns=['PNMOS'])

# 2. Remove unrequired column ( Wfin [nm], alpha )

# X: Wfin, alpha
data_x = data_x.drop('Wfin [nm]', axis=1)
data_x = data_x.drop('alpha', axis=1)

# Y: 
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)

# X_per_cycle
for i in range(num_of_cycle):    
    #print(data_x[i*num_in_cycle:i*num_in_cycle+1])
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values

# X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)

# DATA_Y preprcoessing

# Y_all
for i in range(num_total):
    Y_all[i] = data_y[i:i+1].values
    
# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)
    
print(X_all.shape, Y_all.shape, X_per_cycle.shape, Y_per_cycle.shape)


(10000, 5) (10000, 6) (200, 5) (200, 6)
